## 1. 압축 파일 압축 해제 코드 (압축 폴더가 있을시)
### 라벨링데이터.zip -> data 폴더에 해제
#### data_v3 폴더 사용

In [6]:
import os
import zipfile
import shutil
from tqdm import tqdm

zip_path   = '라벨링데이터.zip'
extract_to = 'data'

# 출력 폴더 생성
os.makedirs(extract_to, exist_ok=True)

def decode_filename(fname: str) -> str:
    """
    zipfile이 cp437로 디코딩한 fname(garbled)을
    1) cp437 바이트로 다시 인코딩 → UTF-8 디코딩 시도
    2) 실패 시 cp949 디코딩 시도
    3) 그마저 실패하면 원본 fname 반환
    """
    raw = fname.encode('cp437', errors='ignore')
    for enc in ('utf-8', 'cp949'):
        try:
            return raw.decode(enc)
        except UnicodeDecodeError:
            continue
    return fname

with zipfile.ZipFile(zip_path, 'r') as z:
    # __MACOSX, .DS_Store, '._'로 시작하는 메타데이터는 제외
    entries = [
        info for info in z.infolist()
        if not (
            info.filename.startswith('__MACOSX/') or
            os.path.basename(info.filename).startswith('._') or
            os.path.basename(info.filename) == '.DS_Store'
        )
    ]

    for info in tqdm(entries, desc='Unzipping files'):
        # 파일명을 깨짐 없이 복원
        safe_name = decode_filename(info.filename)
        out_path   = os.path.join(extract_to, safe_name)

        if info.is_dir():
            os.makedirs(out_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(out_path), exist_ok=True)
            with z.open(info) as src, open(out_path, 'wb') as dst:
                shutil.copyfileobj(src, dst)

Unzipping files: 100%|██████████████████| 35270/35270 [00:03<00:00, 9063.90it/s]


In [5]:
# 테스트 코드

import os
import glob
import json
from tqdm import tqdm

# 1) 처리할 세션 ID 지정
session_id = '40001'

# 2) 분류 폴더 경로, 파일 패턴
input_dir   = 'data_v3/classification'
pattern     = os.path.join(input_dir, f'*_{session_id}_*.json')

# 3) 파일 리스트 정렬 (1→2→3→4→5.json 순)
files = sorted(
    glob.glob(pattern),
    key=lambda p: int(os.path.basename(p).split('_')[-1].split('.')[0])
)

# 4) 진행바로 5개 파일 로드
merged_list = []
for path in tqdm(files, desc=f'Merging classification {session_id}'):
    with open(path, 'r', encoding='utf-8') as f:
        merged_list.append(json.load(f))

# 5) 하나의 객체로 합치기
out_obj = {
    'session_id': session_id,
    '분류': merged_list    # 5개의 JSON 내용이 리스트로
}

# 6) output 폴더에 저장
os.makedirs('output_v3', exist_ok=True)
out_path = os.path.join('output_v3', f'merged_classification_{session_id}.json')
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(out_obj, f, ensure_ascii=False, indent=2)

print(f'✅ {len(files)}개 파일을 병합하여 {out_path}에 저장했습니다.')  


Merging classification 40001: 100%|█████████████| 5/5 [00:00<00:00, 2931.03it/s]

✅ 5개 파일을 병합하여 output_v3/merged_classification_40001.json에 저장했습니다.


#### 중복 내용 제거

In [9]:
import os
import json
from tqdm import tqdm

# 1) 입력·출력 경로
input_path  = 'output_v3/merged_classification_40001.json'
output_path = 'output_v3/merged_classification_40001_cleaned.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 2) 원본 로드
with open(input_path, 'r', encoding='utf-8') as f:
    merged = json.load(f)

# 3) 콘텐츠별로 묶어서 instructions 합치기
grouped = {}  # content_str → {'base': record_without_instr, 'data': [instr_data, ...]}

for entry in tqdm(merged['분류'], desc='Merging duplicates'):
    # entry가 list 안에 dict 하나로 들어 있으면 꺼내고, 아니면 그대로
    rec = entry[0] if isinstance(entry, list) else entry

    content = rec['consulting_content']
    # instructions 구조 안쪽의 data 리스트
    instr_block = rec['instructions'][0]            # e.g. {'tuning_type':'분류','data':[...]}
    data_list   = instr_block.get('data', [])

    if content not in grouped:
        # base에는 중복시켜 쓸 필드들만 남겨 둠
        base = {k: v for k, v in rec.items() if k != 'instructions'}
        grouped[content] = {'base': base, 'data': []}

    # 같은 content에 대한 모든 data_list를 합침
    grouped[content]['data'].extend(data_list)

# 4) 최종 리스트 재구성
cleaned = []
for content, info in grouped.items():
    obj = info['base'].copy()
    # instructions 자리에 합쳐진 data만 넣어서 하나의 튜닝블록으로 만듦
    obj['instructions'] = [{
        'tuning_type': '분류',
        'data': info['data']
    }]
    cleaned.append(obj)

# 5) 저장
out = {
    'session_id': merged.get('session_id'),
    '분류': cleaned
}
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print(f'✅ 중복 제거 및 병합 완료: {len(merged["분류"])} → {len(cleaned)} items')
print('생성된 파일:', output_path)


Merging duplicates: 100%|██████████████████████| 5/5 [00:00<00:00, 61500.06it/s]

✅ 중복 제거 및 병합 완료: 5 → 1 items
생성된 파일: output_v3/merged_classification_40001_cleaned.json


In [17]:
import os
import json
from tqdm import tqdm

# 1) 입력·출력 경로
input_path  = 'output/merged_classification_40001_cleaned.json'
output_path = 'output/merged_classification_40001_cleaned_once.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 2) 로드
with open(input_path, 'r', encoding='utf-8') as f:
    merged = json.load(f)

# 3) 각 레코드별로 중복된 'input' 제거하고, 최상단으로 한 번만 옮기기
for rec in tqdm(merged['분류'], desc='Deduping instruction inputs'):
    data_list = rec['instructions'][0]['data']
    if not data_list:
        continue
    # 3-1) 공통 input 뽑아서 최상단에 추가
    common_input = data_list[0].get('input', '')
    rec['input'] = common_input
    
    # 3-2) 각 data 항목에서 중복된 input 키 제거
    for d in data_list:
        if 'input' in d:
            d.pop('input')

# 4) 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(merged, f, ensure_ascii=False, indent=2)

print(f'✅ 완료: 중복된 input 제거 후 저장 → {output_path}')  


Deduping instruction inputs: 100%|██████████████| 1/1 [00:00<00:00, 9822.73it/s]

✅ 완료: 중복된 input 제거 후 저장 → output/merged_classification_40001_cleaned_once.json


In [18]:
import os
import json
from tqdm import tqdm

# 1) 입력·출력 파일 경로
input_path  = 'output/merged_classification_40001_cleaned_once.json'
output_path = 'output/merged_classification_40001_final.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 2) JSON 로드
with open(input_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 3) '분류' 리스트의 각 레코드에서 'input' 키 제거
for rec in tqdm(data.get('분류', []), desc='Removing top-level input'):
    rec.pop('input', None)

# 4) 결과 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f'✅ 완료: 최종 파일 → {output_path}')


Removing top-level input: 100%|████████████████| 1/1 [00:00<00:00, 12157.40it/s]

✅ 완료: 최종 파일 → output/merged_classification_40001_final.json


## 진짜 시작 - 분류

In [27]:
import os
import glob
import json
from collections import defaultdict
from tqdm import tqdm

# 입력·출력 폴더 설정
INPUT_DIR  = 'data_v3/classification'
OUTPUT_DIR = 'json_merge/classification_merge_output_v3'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1) 파일명 언더스코어 분할로 세션ID별 그룹화
sessions = defaultdict(list)
for fp in glob.glob(os.path.join(INPUT_DIR, '*.json')):
    fname = os.path.basename(fp)
    parts = fname.split('_')
    # parts = ['01', '분류', '40017', '3.json']
    if len(parts) >= 4:
        session_id = parts[2]
        sessions[session_id].append(fp)

# 2) 각 세션별로 5개 파일만 처리
for session_id, fps in tqdm(sessions.items(), desc='세션 처리'):
    if len(fps) != 5:
        continue

    # 3) 번호 순서대로 정렬(1.json → 2.json … 5.json)
    ordered = sorted(
        fps,
        key=lambda p: int(os.path.basename(p).split('_')[-1].split('.')[0])
    )

    # 4) JSON 로드 & consulting_content 기준으로 data 병합
    merged = {}
    for p in ordered:
        arr = json.load(open(p, 'r', encoding='utf-8'))
        rec = arr[0] if isinstance(arr, list) else arr
        content = rec['consulting_content']

        if content not in merged:
            # base 메타( instructions, input 제외 )
            base = {k: v for k, v in rec.items() if k not in ('instructions', 'input')}
            merged[content] = {'base': base, 'data': []}

        # instructions[0]['data']만 합치기
        merged[content]['data'].extend(
            rec['instructions'][0].get('data', [])
        )

    # 5) 최종 오브젝트 리스트 생성
    cleaned = []
    for content, info in merged.items():
        obj = info['base'].copy()
        obj['consulting_content'] = content
        obj['instructions'] = [{
            'tuning_type': '분류',
            'data': info['data']
        }]
        cleaned.append(obj)

    # 6) 세션별로 파일 저장
    out = {
        'session_id': session_id,
        '분류': cleaned
    }
    out_path = os.path.join(OUTPUT_DIR, f'merged_classification_{session_id}_final.json')
    with open(out_path, 'w', encoding='utf-8') as fw:
        json.dump(out, fw, ensure_ascii=False, indent=2)

print("✅ 모든 세션 병합 완료!")  

세션 처리: 100%|███████████████████████████| 3600/3600 [00:03<00:00, 968.93it/s]

✅ 모든 세션 병합 완료!


In [29]:
import os
import glob
import json
from tqdm import tqdm

# 1) 출력 폴더와 파일 패턴 정의
OUTPUT_DIR = 'json_merge/classification_merge_output_v3'
# merged 파일 전체를 대상으로 (분류, 요약, 질의응답 등)
file_pattern = os.path.join(OUTPUT_DIR, 'merged_*_final.json')

# 2) 파일 목록 수집
files = glob.glob(file_pattern)

# 3) 재귀 함수로 모든 'input' 키 제거
def remove_input(obj):
    if isinstance(obj, dict):
        # 최상위 'input' 삭제
        obj.pop('input', None)
        for v in obj.values():
            remove_input(v)
    elif isinstance(obj, list):
        for item in obj:
            remove_input(item)

# 4) 각 파일 처리
for fp in tqdm(files, desc='Removing duplicate inputs'):
    with open(fp, 'r', encoding='utf-8') as f:
        data = json.load(f)
    remove_input(data)
    with open(fp, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

print(f'✅ Removed "input" fields from {len(files)} merged JSON files.')

Removing duplicate inputs: 100%|██████████| 3600/3600 [00:00<00:00, 4235.91it/s]

✅ Removed "input" fields from 3600 merged JSON files.


## 요약 Merge

In [31]:
import os
import glob
import json
from collections import defaultdict
from tqdm import tqdm

# 1) 입력·출력 폴더 설정
INPUT_DIR  = 'data_v3/summary'           # 실제 요약 JSON이 있는 폴더
OUTPUT_DIR = 'json_merge/summary_merge_output_v3'      # 병합 결과를 저장할 폴더
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2) session_id별로 파일 경로 모으기
sessions = defaultdict(list)
for fp in glob.glob(os.path.join(INPUT_DIR, '02_*_*.json')):
    fname = os.path.basename(fp)
    parts = fname.split('_')
    # parts = ['02', '요약', '<session_id>', '1.json']
    if parts[0] != '02':
        continue
    session_id = parts[2]
    sessions[session_id].append(fp)

# 3) 세션별 병합
for session_id, fps in tqdm(sessions.items(), desc='요약 세션 병합'):
    if not fps:
        continue
    
    # 3-1) 번호 순서대로 정렬 (1 → 2 → 3 → 4…)
    ordered = sorted(
        fps,
        key=lambda p: int(os.path.basename(p).split('_')[-1].split('.')[0])
    )
    
    # 3-2) JSON 로드 & 같은 consulting_content 기준으로 data 병합
    merged = {}
    for p in ordered:
        with open(p, 'r', encoding='utf-8') as f:
            arr = json.load(f)
        rec = arr[0] if isinstance(arr, list) else arr
        
        # 공통된 상담 대화 원문으로 그룹화
        content = rec.get('consulting_content', '')
        if content not in merged:
            # instructions, input 제외한 기본 메타만 보관
            base = {k: v for k, v in rec.items() if k not in ('instructions', 'input')}
            merged[content] = {'base': base, 'data': []}
        
        # instructions[0]['data'] 병합
        merged[content]['data'].extend(
            rec['instructions'][0].get('data', [])
        )
    
    # 3-3) 최종 리스트 생성
    cleaned = []
    for content, info in merged.items():
        obj = info['base'].copy()
        obj['consulting_content'] = content
        obj['instructions'] = [{
            'tuning_type': '요약',
            'data': info['data']
        }]
        cleaned.append(obj)
    
    # 4) JSON으로 저장
    out = {
        'session_id': session_id,
        '요약': cleaned
    }
    out_path = os.path.join(OUTPUT_DIR, f'merged_summary_{session_id}.json')
    with open(out_path, 'w', encoding='utf-8') as fw:
        json.dump(out, fw, ensure_ascii=False, indent=2)

print('✅ 요약 폴더 병합 완료 →', OUTPUT_DIR)

요약 세션 병합: 100%|█████████████████████| 3600/3600 [00:02<00:00, 1248.26it/s]

✅ 요약 폴더 병합 완료 → json_merge/summary_merge_output_v3


In [33]:
import os
import glob
import json
from tqdm import tqdm

# 1) 병합된 요약 파일들이 있는 폴더
INPUT_DIR = 'json_merge/summary_merge_output_v3'
pattern   = os.path.join(INPUT_DIR, 'merged_summary_*.json')

# 2) 각 파일마다 'input' 키 제거
for filepath in tqdm(glob.glob(pattern), desc='Removing inputs in summary'):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # '요약' 리스트 내부 각 레코드의 'input' 삭제
    for rec in data.get('요약', []):
        rec.pop('input', None)
        # instructions.data 내부의 'input'도 삭제
        for inst in rec.get('instructions', []):
            for item in inst.get('data', []):
                item.pop('input', None)

    # 3) 변경 내용 덮어쓰기 저장
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ 모든 merged_summary 파일에서 'input' 제거 완료")  

Removing inputs in summary: 100%|█████████| 3600/3600 [00:00<00:00, 5254.49it/s]

✅ 모든 merged_summary 파일에서 'input' 제거 완료


## 질의응답

In [35]:
import os
import glob
import json
from collections import defaultdict
from tqdm import tqdm

# 1) 입력·출력 폴더 설정
INPUT_DIR  = 'data_v3/qa'   # 질의응답 JSON 파일들이 있는 폴더
OUTPUT_DIR = 'json_merge/qa_merge_output_v3'      # 병합 결과를 저장할 폴더
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2) session_id별로 파일 경로 모으기
sessions = defaultdict(list)
for fp in glob.glob(os.path.join(INPUT_DIR, '03_*_*.json')):
    fname = os.path.basename(fp)
    parts = fname.split('_')
    # 파일명 예시: '03_질의응답_44017_1.json'
    if parts[0] != '03':
        continue
    session_id = parts[2]
    sessions[session_id].append(fp)

# 3) 세션별 병합
for session_id, fps in tqdm(sessions.items(), desc='질의응답 세션 병합'):
    if not fps:
        continue
    
    # 3-1) 번호 순서대로 정렬 (1 → 2 → 3 → …)
    ordered = sorted(
        fps,
        key=lambda p: int(os.path.basename(p).split('_')[-1].split('.')[0])
    )
    
    # 3-2) JSON 로드 & 같은 consulting_content 기준으로 data 병합
    merged = {}
    for p in ordered:
        with open(p, 'r', encoding='utf-8') as f:
            arr = json.load(f)
        rec = arr[0] if isinstance(arr, list) else arr
        content = rec.get('consulting_content', '')
        
        if content not in merged:
            # instructions/input 제외한 메타데이터만 저장
            base = {k: v for k, v in rec.items() if k not in ('instructions', 'input')}
            merged[content] = {'base': base, 'data': []}
        
        # instructions[0]['data'] 병합
        merged[content]['data'].extend(
            rec['instructions'][0].get('data', [])
        )
    
    # 3-3) 최종 오브젝트 리스트 생성
    cleaned = []
    for content, info in merged.items():
        obj = info['base'].copy()
        obj['consulting_content'] = content
        obj['instructions'] = [{
            'tuning_type': '질의응답',
            'data': info['data']
        }]
        cleaned.append(obj)
    
    # 4) JSON으로 저장
    out = {
        'session_id': session_id,
        '질의응답': cleaned
    }
    out_path = os.path.join(OUTPUT_DIR, f'merged_qna_{session_id}.json')
    with open(out_path, 'w', encoding='utf-8') as fw:
        json.dump(out, fw, ensure_ascii=False, indent=2)

print('✅ 질의응답 폴더 병합 완료 →', OUTPUT_DIR)

# 5) 병합된 파일에서 중복된 'input' 필드 제거
pattern = os.path.join(OUTPUT_DIR, 'merged_qna_*.json')
for filepath in tqdm(glob.glob(pattern), desc='Removing inputs in QnA'):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # 최상위 'input' 삭제
    data.pop('input', None)
    for rec in data.get('질의응답', []):
        rec.pop('input', None)
        for inst in rec.get('instructions', []):
            for item in inst.get('data', []):
                item.pop('input', None)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ QnA merged 파일에서 'input' 제거 완료")

질의응답 세션 병합: 100%|█████████████████| 3533/3533 [00:02<00:00, 1761.69it/s]


✅ 질의응답 폴더 병합 완료 → json_merge/qa_merge_output_v3


Removing inputs in QnA: 100%|█████████████| 3533/3533 [00:00<00:00, 6267.38it/s]

✅ QnA merged 파일에서 'input' 제거 완료


# 최종 Merge (분류 + 요약 + 질의응답)

In [37]:
import os
import glob
import json
from collections import defaultdict
from tqdm import tqdm

# 1) 폴더 경로 설정
CLASS_DIR   = 'json_merge/classification_merge_output_v3' # 분류 파일 보관 폴더
SUMMARY_DIR = 'json_merge/summary_merge_output_v3'        # 요약 파일 보관 폴더
QNA_DIR     = 'json_merge/qa_merge_output_v3'            # 질의응답 파일 보관 폴더
FINAL_DIR   = 'json_merge/integration_data_v3'          # 최종 머지 파일 저장 폴더
os.makedirs(FINAL_DIR, exist_ok=True)

# 2) 세션ID별로 파일 경로 매핑
sessions = defaultdict(dict)
for path in glob.glob(os.path.join(CLASS_DIR, 'merged_classification_*_final.json')):
    sid = os.path.basename(path).split('_')[2]
    sessions[sid]['class'] = path
for path in glob.glob(os.path.join(SUMMARY_DIR, 'merged_summary_*.json')):
    sid = os.path.basename(path).split('_')[2].split('.')[0]
    sessions[sid]['summary'] = path
for path in glob.glob(os.path.join(QNA_DIR, 'merged_qna_*.json')):
    sid = os.path.basename(path).split('_')[2].split('.')[0]
    sessions[sid]['qna'] = path

# 3) 세션별로 합치기
for sid, files in tqdm(sessions.items(), desc='최종 세션 통합'):
    if not all(k in files for k in ('class','summary','qna')):
        continue  # 세 가지 모두 있어야 처리

    # 3-1) 각각 로드
    cls = json.load(open(files['class'],   'r', encoding='utf-8'))
    smm = json.load(open(files['summary'], 'r', encoding='utf-8'))
    qna = json.load(open(files['qna'],     'r', encoding='utf-8'))

    # 3-2) 공통 대화 원문 추출
    content = cls['분류'][0]['consulting_content']

    # 3-3) 모든 instructions.data 항목 모으기
    all_inst = []
    for rec in cls['분류']:
        for blk in rec['instructions']:
            all_inst.extend(blk.get('data', []))
    for rec in smm['요약']:
        for blk in rec['instructions']:
            all_inst.extend(blk.get('data', []))
    for rec in qna['질의응답']:
        for blk in rec['instructions']:
            all_inst.extend(blk.get('data', []))

    # 3-4) instruction_id 오름차순 정렬 + 중복 제거
    all_inst_sorted = sorted(all_inst, key=lambda x: x.get('instruction_id', 0))
    seen = set()
    dedup = []
    for inst in all_inst_sorted:
        iid = inst.get('instruction_id')
        if iid not in seen:
            seen.add(iid)
            dedup.append(inst)

    # 3-5) 최종 구조 생성
    final = {
        'session_id': sid,
        'consulting_content': content,
        'instructions': dedup
    }

    # 4) 저장
    out_path = os.path.join(FINAL_DIR, f'final_merged_{sid}.json')
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(final, f, ensure_ascii=False, indent=2)

print(f'✅ 최종 통합 완료 → {FINAL_DIR}/final_merged_<session_id>.json')  

최종 세션 통합: 100%|█████████████████████| 3600/3600 [00:01<00:00, 1867.62it/s]

✅ 최종 통합 완료 → json_merge/integration_data_v3/final_merged_<session_id>.json


In [39]:
import os
import glob
import json
from tqdm import tqdm

# 1) 디렉토리 설정
CLASS_DIR      = 'json_merge/classification_merge_output_v3'   # merged_classification_XXX_final.json 들이 있는 폴더
FINAL_DIR      = 'json_merge/integration_data_v3'            # 지금까지 만든 final_merged_XXX.json 들이 있는 폴더
OUTPUT_DIR     = 'json_merge/integration_data_v3_with_meta'  # 메타 정보가 추가된 최종 파일들을 저장할 폴더
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2) 모든 classification 파일 순회
class_files = glob.glob(os.path.join(CLASS_DIR, 'merged_classification_*_final.json'))
for class_fp in tqdm(class_files, desc='Merging classification meta into final'):
    # 세션 ID 추출 (파일명 끝에서 두 번째 언더바 사이)
    session_id = os.path.basename(class_fp).split('_')[-2]
    
    # 3) classification 메타 불러오기
    class_data = json.load(open(class_fp, 'r', encoding='utf-8'))
    # 분류 리스트의 첫 번째 원소에 담긴 메타 정보만 추출
    meta_item = class_data['분류'][0]
    meta_fields = {
        'source':               meta_item.get('source'),
        'source_id':            meta_item.get('source_id'),
        'consulting_category':  meta_item.get('consulting_category'),
        'consulting_time':      meta_item.get('consulting_time'),
        'consulting_turns':     meta_item.get('consulting_turns'),
        'consulting_length':    meta_item.get('consulting_length'),
    }
    
    # 4) 기존 final 파일 불러오기
    final_fp = os.path.join(FINAL_DIR, f'final_merged_{session_id}.json')
    if not os.path.exists(final_fp):
        # final 파일이 없으면 건너뜀
        continue
    final_data = json.load(open(final_fp, 'r', encoding='utf-8'))
    
    # 5) 메타를 최상단으로 병합
    # 순서를 보장하려면 새로운 dict 생성
    new_final = {}
    new_final['session_id'] = final_data.get('session_id')
    # classification 메타 삽입
    new_final.update(meta_fields)
    # 나머지 기존 final 데이터 삽입
    # (consulting_content, instructions 등)
    for k, v in final_data.items():
        if k == 'session_id':
            continue
        new_final[k] = v
    
    # 6) 저장
    out_fp = os.path.join(OUTPUT_DIR, f'final_merged_{session_id}_with_meta.json')
    with open(out_fp, 'w', encoding='utf-8') as fw:
        json.dump(new_final, fw, ensure_ascii=False, indent=2)

print('✅ Classification 메타가 추가된 final 파일 생성 완료.')

Merging classification meta into final: 100%|█| 3600/3600 [00:00<00:00, 3652.77i

✅ Classification 메타가 추가된 final 파일 생성 완료.
